# DATA 601 Final Project
# Severe Thunderstorm Climatology vs Sunspot Number 2019-2020
## Python weather analysis application that compares the occurrence of severe thunderstorm activity occurring over the continental Unites States to daily international sunspot number measurements.

In [1]:
#Load modules and invoke WxData API
from __future__ import print_function, division
import numpy as np
import pandas as pd 
import sys
import requests
import xmltodict

## Data Cleaning: Define a function that builds a storm report listing for each semi-monthly period between November 2019 and April 2020.

In [2]:
#Function that builds a storm report listing for each semi-monthly period between November 2019 and April 2020.
def dataframe_strmrpt(sr_month):
    responsewx = requests.get(sr_month)
    wxdata_sr = responsewx.content
    xmlDict = xmltodict.parse(wxdata_sr) #Build storm report dictionary from XML files
    
    #Initiate lists for each storm report attribuite
    id = [] #Report ID of each severe thunderstorm event
    mag = [] #Thunderstorm wind gust magnitude in units of miles per hour (MPH)
    state = [] #US state of occurrence of severe thunderstorm event
    date = [] #Date and time of occurrence of severe thunderstorm event
    
    if xmlDict['response']['data'].get('storm-report')!=None: #Test for the existence of storm reports for the period of interest
        print("The key 'storm-report' is present.\n") 
        #If storm reports are present, build the storm report list
        for stormreports in xmlDict['response']['data']['storm-report']:
            id.append(stormreports['id'])
            mag.append(stormreports['magnitude'])
            state.append(stormreports['state'])
            date.append(stormreports['event_local_time'])

        # Reversing a list using reverse() 
        def Reverse(lst): 
            lst.reverse() 
            return lst 
        #Build Pandas series for each storm report attribuite
        sr_id = pd.Series(Reverse(id))
        sr_date = pd.Series(Reverse(date))
        sr_state = pd.Series(Reverse(state))
        sr_mag = pd.Series(Reverse(mag))
        #Build a Pandas dataframe from each individual storm report series
        #Extract maximum thunderstorm wind measurement in units of miles per hour (MPH) for each semi-monthly period
        wxdata_df = pd.DataFrame({'ID':sr_id,'Date':sr_date,'State':sr_state,'Magnitude':sr_mag})
        #Clean dataset, remove MPH label from string, save each wind report as an integer 
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\M', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\E', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\PH', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].str.replace(r'\.00', '')
        wxdata_df['Magnitude'] = wxdata_df['Magnitude'].astype(int)
        maxmag = np.amax(wxdata_df['Magnitude']) #Extract maximum wind gust magnitude for semi-monthly period
        strmrpt_nr = len(wxdata_df.Magnitude) #Count number of storm reports for semi-monthly period
    
    else: 
        #If no storm reports exist, set analysis parameters = 0 and the dataframe = None
        print("The key 'storm-report' does not exist in the dictionary.") 
        wxdata_df = 'None'
        strmrpt_nr = 0
        maxmag = 0
    return wxdata_df #Return storm report dataframe and descriptive statistics

## API Data Collection: Invoke Requests to call the WxData Storm Reports API Builder.

In [6]:
#Invoke requests to call the WxData Storm Reports API Builder. Execute an API call for each semi-monthly period between November 2019 and April 2020.
#See https://wxdata.com/storm-reports-api-format#event
nov2019_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-01T00:00:00&enddate=2019-11-16T00:00:00&event=tstm+wnd+gst"
nov2019_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-01T00:00:00&enddate=2019-11-16T00:00:00&event=marine+tstm+wind"
nov2019_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-16T00:00:00&enddate=2019-12-01T00:00:00&event=tstm+wnd+gst"
nov2019_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-11-16T00:00:00&enddate=2019-12-01T00:00:00&event=tstm+wnd+gst"

df_wxdata_nov2019e_twg = dataframe_strmrpt(nov2019_1_twg)
df_wxdata_nov2019e_twg.to_csv(r'data\wxdata_nov2019e_twg.csv')
print("wxdata_nov2019e_twg.csv saved")
df_wxdata_nov2019e_mtw = dataframe_strmrpt(nov2019_1_mtw)
df_wxdata_nov2019e_mtw.to_csv(r'data\wxdata_nov2019e_mtw.csv')
print("wxdata_nov2019e_mtw.csv saved")
df_wxdata_nov2019l_twg = dataframe_strmrpt(nov2019_2_twg)
df_wxdata_nov2019l_twg.to_csv(r'data\wxdata_nov2019l_twg.csv')
print("wxdata_nov2019l_twg.csv saved")
df_wxdata_nov2019l_mtw = dataframe_strmrpt(nov2019_2_mtw)
df_wxdata_nov2019l_mtw.to_csv(r'data\wxdata_nov2019l_mtw.csv')
print("wxdata_nov2019l_mtw.csv saved")


The key 'storm-report' is present.

wxdata_nov2019e_twg.csv saved
The key 'storm-report' is present.

wxdata_nov2019e_mtw.csv saved
The key 'storm-report' is present.

wxdata_nov2019l_twg.csv saved
The key 'storm-report' is present.

wxdata_nov2019l_mtw.csv saved


In [3]:
dec2019_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-01T00:00:00&enddate=2019-12-16T00:00:00&event=tstm+wnd+gst"
dec2019_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-01T00:00:00&enddate=2019-12-16T00:00:00&event=marine+tstm+wind"
dec2019_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-16T00:00:00&enddate=2020-01-01T00:00:00&event=tstm+wnd+gst"
dec2019_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2019-12-16T00:00:00&enddate=2020-01-01T00:00:00&event=marine+tstm+wind"

df_wxdata_dec2019e_twg = dataframe_strmrpt(dec2019_1_twg)
df_wxdata_dec2019e_twg.to_csv(r'data\df_wxdata_dec2019e_twg.csv')
print("wxdata_dec2019e_twg.csv saved")
df_wxdata_dec2019e_mtw = dataframe_strmrpt(dec2019_1_mtw)
df_wxdata_dec2019e_mtw.to_csv(r'data\df_wxdata_dec2019e_mtw.csv')
print("wxdata_dec2019e_mtw.csv saved")
df_wxdata_dec2019l_twg = dataframe_strmrpt(dec2019_2_twg)
df_wxdata_dec2019l_twg.to_csv(r'data\df_wxdata_dec2019l_twg.csv')
print("wxdata_dec20191_twg.csv saved")
df_wxdata_dec2019l_mtw = dataframe_strmrpt(dec2019_2_mtw)
df_wxdata_dec2019l_mtw.to_csv(r'data\df_wxdata_dec2019l_mtw.csv')
print("wxdata_dec20191_mtw.csv saved")

The key 'storm-report' is present.

wxdata_dec2019e_twg.csv saved
The key 'storm-report' is present.

wxdata_dec2019e_mtw.csv saved
The key 'storm-report' is present.

wxdata_dec20191_twg.csv saved
The key 'storm-report' is present.

wxdata_dec20191_mtw.csv saved


In [4]:
jan2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-01-01T00:00:00&enddate=2020-01-16T00:00:00&event=tstm+wnd+gst"
jan2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-01-01T00:00:00&enddate=2020-01-16T00:00:00&event=marine+tstm+wind"
jan2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-01-16T00:00:00&enddate=2020-02-01T00:00:00&event=marine+tstm+wind"

df_wxdata_jan2020e_twg = dataframe_strmrpt(jan2020_1_twg)
df_wxdata_jan2020e_twg.to_csv(r'data\df_wxdata_jan2020e_twg.csv')
print("wxdata_jan2020e_twg.csv saved")
df_wxdata_jan2020e_mtw = dataframe_strmrpt(jan2020_1_mtw)
df_wxdata_jan2020e_mtw.to_csv(r'data\df_wxdata_jan2020e_mtw.csv')
print("wxdata_jan2020e_mtw.csv saved")
df_wxdata_jan2020l_mtw = dataframe_strmrpt(jan2020_2_mtw)
df_wxdata_jan2020l_mtw.to_csv(r'data\df_wxdata_jan2020l_mtw.csv')
print("wxdata_jan2020l_mtw.csv saved")

The key 'storm-report' is present.

wxdata_jan2020e_twg.csv saved
The key 'storm-report' is present.

wxdata_jan2020e_mtw.csv saved
The key 'storm-report' is present.

wxdata_jan2020l_mtw.csv saved


In [5]:
feb2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-01T00:00:00&enddate=2020-02-15T00:00:00&event=tstm+wnd+gst"
feb2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-01T00:00:00&enddate=2020-02-15T00:00:00&event=marine+tstm+wind"
feb2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-15T00:00:00&enddate=2020-03-01T00:00:00&event=tstm+wnd+gst"
feb2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-02-15T00:00:00&enddate=2020-03-01T00:00:00&event=marine+tstm+wind"

df_wxdata_feb2020e_twg = dataframe_strmrpt(feb2020_1_twg)
df_wxdata_feb2020e_twg.to_csv(r'data\df_wxdata_feb2020e_twg.csv')
print("wxdata_feb2020e_twg.csv saved")
df_wxdata_feb2020e_mtw = dataframe_strmrpt(feb2020_1_mtw)
df_wxdata_feb2020e_mtw.to_csv(r'data\df_wxdata_feb2020e_mtw.csv')
print("wxdata_feb2020e_mtw.csv saved")
df_wxdata_feb2020l_twg = dataframe_strmrpt(feb2020_2_twg)
df_wxdata_feb2020l_twg.to_csv(r'data\df_wxdata_feb2020l_twg.csv')
print("wxdata_feb2020l_twg.csv saved")
df_wxdata_feb2020l_mtw = dataframe_strmrpt(feb2020_2_mtw)
df_wxdata_feb2020l_mtw.to_csv(r'data\df_wxdata_feb2020l_mtw.csv')
print("wxdata_feb2020l_mtw.csv saved")



The key 'storm-report' is present.

wxdata_feb2020e_twg.csv saved
The key 'storm-report' is present.

wxdata_feb2020e_mtw.csv saved
The key 'storm-report' is present.

wxdata_feb2020l_twg.csv saved
The key 'storm-report' is present.

wxdata_feb2020l_mtw.csv saved


In [3]:
mar2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-01T00:00:00&enddate=2020-03-16T00:00:00&event=tstm+wnd+gst"
mar2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-01T00:00:00&enddate=2020-03-16T00:00:00&event=marine+tstm+wind"
mar2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-16T00:00:00&enddate=2020-04-01T00:00:00&event=tstm+wnd+gst"
mar2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-03-16T00:00:00&enddate=2020-04-01T00:00:00&event=marine+tstm+wind"

df_wxdata_mar2020e_twg = dataframe_strmrpt(mar2020_1_twg)
df_wxdata_mar2020e_twg.to_csv(r'data\df_wxdata_mar2020e_twg.csv')
print("wxdata_mar2020e_twg.csv saved")
df_wxdata_mar2020e_mtw = dataframe_strmrpt(mar2020_1_mtw)
df_wxdata_mar2020e_mtw.to_csv(r'data\df_wxdata_mar2020e_mtw.csv')
print("wxdata_mar2020e_mtw.csv saved")
df_wxdata_mar2020l_twg = dataframe_strmrpt(mar2020_2_twg)
df_wxdata_mar2020l_twg.to_csv(r'data\df_wxdata_mar2020l_twg.csv')
print("wxdata_mar2020l_twg.csv saved")
df_wxdata_mar2020l_mtw = dataframe_strmrpt(mar2020_2_mtw)
df_wxdata_mar2020l_mtw.to_csv(r'data\df_wxdata_mar2020l_mtw.csv')
print("wxdata_mar2020l_mtw.csv saved")


The key 'storm-report' is present.

wxdata_mar2020e_twg.csv saved
The key 'storm-report' is present.

wxdata_mar2020e_mtw.csv saved
The key 'storm-report' is present.

wxdata_mar2020l_twg.csv saved
The key 'storm-report' is present.

wxdata_mar2020l_mtw.csv saved


In [4]:
apr2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-01T00:00:00&enddate=2020-04-16T00:00:00&event=tstm+wnd+gst"
apr2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-01T00:00:00&enddate=2020-04-16T00:00:00&event=marine+tstm+wind"
apr2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-16T00:00:00&enddate=2020-04-30T23:00:00&event=tstm+wnd+gst"
apr2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-04-16T00:00:00&enddate=2020-04-30T23:00:00&event=marine+tstm+wind"

df_wxdata_apr2020e_twg = dataframe_strmrpt(apr2020_1_twg)
df_wxdata_apr2020e_twg.to_csv(r'data\df_wxdata_apr2020e_twg.csv')
print("wxdata_apr2020e_twg.csv saved")
df_wxdata_apr2020e_mtw = dataframe_strmrpt(apr2020_1_mtw)
df_wxdata_apr2020e_mtw.to_csv(r'data\df_wxdata_apr2020e_mtw.csv')
print("wxdata_apr2020e_mtw.csv saved")
df_wxdata_apr2020l_twg = dataframe_strmrpt(apr2020_2_twg)
df_wxdata_apr2020l_twg.to_csv(r'data\df_wxdata_apr2020l_twg.csv')
print("wxdata_apr2020l_twg.csv saved")
df_wxdata_apr2020l_mtw = dataframe_strmrpt(apr2020_2_mtw)
df_wxdata_apr2020l_mtw.to_csv(r'data\df_wxdata_apr2020l_mtw.csv')
print("wxdata_apr2020l_mtw.csv saved")

The key 'storm-report' is present.

wxdata_apr2020e_twg.csv saved
The key 'storm-report' is present.

wxdata_apr2020e_mtw.csv saved
The key 'storm-report' is present.

wxdata_apr2020l_twg.csv saved
The key 'storm-report' is present.

wxdata_apr2020l_mtw.csv saved


In [4]:
#Invoke requests to call the WxData Storm Reports API Builder. Execute an API call for each semi-monthly period between November 2019 and April 2020.
#See https://wxdata.com/storm-reports-api-format#event
#nov2020_1_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-01T00:00:00&enddate=2020-11-16T00:00:00&event=tstm+wnd+gst"
nov2020_1_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-01T00:00:00&enddate=2020-11-16T00:00:00&event=marine+tstm+wind"
nov2020_2_twg = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-16T00:00:00&enddate=2020-12-01T00:00:00&event=tstm+wnd+gst"
nov2020_2_mtw = "https://wxdata.com/cgi-bin/storm-report-api.pl?version=1.0&format=xml&key=2sGNfZOvAT6BSCY&secret=U6St4Q5fNO1enc8&date=2020-11-16T00:00:00&enddate=2020-12-01T00:00:00&event=tstm+wnd+gst"

#df_wxdata_nov2020e_twg = dataframe_strmrpt(nov2020_1_twg)
#df_wxdata_nov2020e_twg.to_csv(r'data\wxdata_nov2020e_twg.csv')
#print("wxdata_nov2020e_twg.csv saved")
df_wxdata_nov2020e_mtw = dataframe_strmrpt(nov2020_1_mtw)
df_wxdata_nov2020e_mtw.to_csv(r'data\wxdata_nov2020e_mtw.csv')
print("wxdata_nov2020e_mtw.csv saved")
df_wxdata_nov2020l_twg = dataframe_strmrpt(nov2020_2_twg)
df_wxdata_nov2020l_twg.to_csv(r'data\wxdata_nov2020l_twg.csv')
print("wxdata_nov2020l_twg.csv saved")
df_wxdata_nov2020l_mtw = dataframe_strmrpt(nov2020_2_mtw)
df_wxdata_nov2020l_mtw.to_csv(r'data\wxdata_nov2020l_mtw.csv')
print("wxdata_nov2020l_mtw.csv saved")


The key 'storm-report' is present.

wxdata_nov2020e_mtw.csv saved
The key 'storm-report' is present.

wxdata_nov2020l_twg.csv saved
The key 'storm-report' is present.

wxdata_nov2020l_mtw.csv saved
